In [1]:
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
import tensorflow as tf

from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

"""loaded = tf.saved_model.load('modelpb')
infer = loaded.signatures['serving_default']

f = tf.function(infer).get_concrete_function(input_1=tf.TensorSpec(shape=[None, 150, 150, 3], dtype=tf.float32))
f2 = convert_variables_to_constants_v2(f)
graph_def = f2.graph.as_graph_def()

# Export frozen graph
with tf.io.gfile.GFile('frozen_graph.pb', 'wb') as f:
   f.write(graph_def.SerializeToString())"""

"loaded = tf.saved_model.load('modelpb')\ninfer = loaded.signatures['serving_default']\n\nf = tf.function(infer).get_concrete_function(input_1=tf.TensorSpec(shape=[None, 150, 150, 3], dtype=tf.float32))\nf2 = convert_variables_to_constants_v2(f)\ngraph_def = f2.graph.as_graph_def()\n\n# Export frozen graph\nwith tf.io.gfile.GFile('frozen_graph.pb', 'wb') as f:\n   f.write(graph_def.SerializeToString())"

In [3]:
prototxtPath = os.path.sep.join(["/home/soundarzozm/Desktop/mask_detector", "deploy.prototxt"])
weightsPath = os.path.sep.join(["/home/soundarzozm/Desktop/mask_detector", "res10_300x300_ssd_iter_140000.caffemodel"])

net = cv2.dnn.readNet(prototxtPath, weightsPath)

model = load_model("model.h5")

In [4]:
image = cv2.imread("try1.jpg")
orig = image.copy()
(h, w) = image.shape[:2]

blob = cv2.dnn.blobFromImage(image, 1.0, (256, 256), (104.0, 177.0, 123.0))

net.setInput(blob)
detections = net.forward()

In [5]:
detections

array([[[[0.        , 1.        , 0.9861102 , ..., 0.6584748 ,
          0.3973991 , 0.8417076 ],
         [0.        , 1.        , 0.64489466, ..., 0.6347736 ,
          0.8550958 , 0.78778493],
         [0.        , 1.        , 0.31283584, ..., 0.5941841 ,
          0.6268668 , 0.73568445],
         ...,
         [0.        , 1.        , 0.11990744, ..., 0.49326447,
          0.7980148 , 0.54044634],
         [0.        , 1.        , 0.11978975, ..., 0.42748797,
          0.272643  , 0.4832241 ],
         [0.        , 1.        , 0.11978081, ..., 0.31856924,
          0.20144944, 0.3792904 ]]]], dtype=float32)

In [6]:
for i in range(0, detections.shape[2]):

    confidence = detections[0, 0, i, 2]

    if confidence > 0.3:

        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")


        (startX, startY) = (max(0, startX), max(0, startY))
        (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
        
        face = image[startY:endY, startX:endX]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (256, 256))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)


        mask = model.predict(face)[0][0]
        no_mask = 1 - mask
        

        label = "Mask" if mask < no_mask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

        label = "{}: {:.2f}%".format(label, max(mask, no_mask) * 100)

        cv2.putText(image, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)

cv2.imshow("Output", image)
cv2.imwrite("try1mod.jpg", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

113